In [50]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep
import re
import pickle
from numpy.random import randint

In [51]:
def scrape_30_listings(driver):
    # Loop through all 30 entries per page
    for post in range(29):
        
        # Pull job details from the list of jobs on left side of browser window
        listing_str = driver.find_element_by_xpath("(//li[@class='jl selected'])").text
        listing_int = listing_str.split("\n")
        
        # Clean up the data so order of elements is always ['Rating','Job_title','Company','Location', 'Salary_range', 'Status', 'Job_description', 'Company_info'] 
        if not bool(re.search(r'\d+.*', listing_int[0])): 
            listing_int.insert(0, "") # If no star rating available, insert a blank
        listing_lst = listing_int[0:2] + listing_int[2].split(" – ") + listing_int[3:] # Split company and location into 2 fields
        if 'EASY APPLY' in listing_lst: listing_lst.remove('EASY APPLY') # Remove "Easy Apply"
        if len(listing_lst) >= 5:
                if not bool(re.search(r'\D\d+[\w]\s*[-]\D\s*\d+\w+.*', listing_lst[4])): 
                    listing_lst.insert(4, "") # If no estimated salary available, insert a blank
        
        # Pull full text for job description from right side of browser window
        try:
            job_description = driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc'])").text
        except:
            job_description = '-'
        listing_lst.append(job_description)
        
        # Pull bulleted lists from job description from right side of browser window
        try:
            job_description_bullets = driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[1])").text
        except:
            job_description_bullets = '-' 
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[2])").text
        except:
            pass  
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[3])").text
        except:
            pass
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[3])").text
        except:
            pass
        listing_lst.append(job_description_bullets)
        
        # If the company tab exists for this listing, click on it and scrape company info
        try:
            driver.find_element_by_xpath("(//li[@data-tab-type='overview'])").click()
            sleep(randint(1000,3000)/1000)
            company_str = driver.find_element_by_xpath("(//div[@id='EmpBasicInfo'])").text
        except:
            company_str = '-'
        listing_lst.append(company_str)
        
        # Get the link to the listing
        element = masked_driver.find_element_by_xpath("(//li[@class='jl selected']/div/a)")
        link = element.get_attribute('href')
        listing_lst.append(link)
        
        # Turn the data into a pandas Dataframe and add that df to the list, 'frames'
        listing_df = pd.DataFrame([listing_lst])
        frames.append(listing_df)
        
        # Click on next job posting from the list on left side of browser window
        driver.find_element_by_xpath("(//li[@class='jl'])[position()=" + str(post+1) + "]").click()
        sleep(randint(1000,3000)/1000)
        
    # Stack each entry into one large dataframe    
    features_df = pd.concat(frames)
    return features_df


In [52]:
def go_to_next_page(driver):
    driver.find_element_by_xpath("(//li[@class='next'])").click()
    sleep(randint(3000,5000)/1000)

In [53]:
def collect_mult_pages(masked_driver):
    
    scrape_30_listings(masked_driver)
    
    for i in range(29):
        go_to_next_page(masked_driver)
        sleep(randint(500,1000)/1000)
        x = scrape_30_listings(masked_driver)
        sleep(randint(500,1000)/1000)

    return x

In [54]:
# # Used to test proxy id's

# proxy = '95.181.35.30:40804'
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % proxy)
# masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
# masked_driver.get("https://whatismyipaddress.com/")

In [58]:
proxy_1 = '139.59.177.41:3128'
proxy_2 = '200.122.209.78:47220'
proxy_3 = '88.247.9.171:58985'
proxy_4 = '95.181.35.30:40804'

In [59]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % proxy_1)
masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
search_phrase='chef'
masked_driver.get("https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=" + search_phrase + "&sc.keyword=" + search_phrase + "&locT=N&locId=1&fromAge=1&jobType=")


In [61]:
chef = collect_mult_pages(masked_driver)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"(//li[@class='jl selected'])"}
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.14.0 x86_64)


In [64]:
chef = pd.concat(frames)
pickle.dump(chef, open( "chef_posts_long.pkl", "wb" ) )

In [62]:
len(frames)

854

In [ ]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % proxy_2)
# masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
# search_phrase='machine+learning'
# masked_driver.get("https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=" + search_phrase + "&sc.keyword=" + search_phrase + "&locT=N&locId=1&fromAge=1&jobType=")


In [ ]:
# frames=[]
# machine_learning = collect_mult_pages(masked_driver)
# pickle.dump(machine_learning, open( "machine_learning_posts.pkl", "wb" ) )

In [16]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % proxy_3)
# masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
# search_phrase='artificial+intelligence'
# masked_driver.get("https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=" + search_phrase + "&sc.keyword=" + search_phrase + "&locT=N&locId=1&fromAge=1&jobType=")


In [ ]:
# frames=[]
# artificial_intelligence = collect_mult_pages(masked_driver)
# pickle.dump(artificial_intelligence, open( "artificial_intelligence_posts.pkl", "wb" ) )

In [ ]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % proxy_4)
# masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
# search_phrase='data+analyst'
# masked_driver.get("https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=" + search_phrase + "&sc.keyword=" + search_phrase + "&locT=N&locId=1&fromAge=1&jobType=")


In [ ]:
# frames=[]
# data_analyst = collect_mult_pages(masked_driver)
# pickle.dump(data_analyst, open( "data_analyst_posts.pkl", "wb" ) )